# Argovis explore vertical structure of ocean fields

This notebook contains some examples and is a work in progress: code will transition from here to other notebooks. Public consumers should focus on our educational notebooks, starting with [Intro to Argovis](https://github.com/argovis/demo_notebooks/blob/main/Intro_to_Argovis.ipynb).

In [ ]:
from argovisHelpers import helpers as avh
import datetime #, pandas, matplotlib, scipy, numpy
import numpy as np
import matplotlib.pyplot as plt
#from matplotlib import cm
#import cartopy.crs as ccrs

from Argovis_tasks_helpers import get_route, get_profiles_in_regions_and_horiz_ave

API_KEY=''

In [ ]:
# API call showing collections available for each route

#### parameters the user needs to select
# user selects list of collections to use for the plot

#### for bgc variables we have gridded time mean fields
collections = ['argo', 'grids/glodap',
              ] # we compare argo profiles with a gridded product

### for temperature and salinity only we have a monthly product
# collections = ['argo', 'grids/rg09',
#               ] # we compare argo profiles with a gridded product

# show list of variables available
for icollection in collections:
    try:
        print(avh.query(icollection+'/vocabulary', options={'parameter': 'data'}, verbose='true',apikey=API_KEY, apiroot=get_route(icollection)) )
    except:
        print('No data parameter for vocabulary query')
    try:
        print(avh.query(icollection+'/vocabulary', verbose='true',apikey=API_KEY, apiroot=get_route(icollection)) )
    except:
        print('Needs data parameter for vocabulary query')
    
# define name of variable of interest for each product (first show list of all variables available for each collection)
varname     = ['doxy', 'oxygen']
varname_qc  = [',1', '']
vartitle    = 'Oxygen, umol/kg'

# varname     = ['salinity', 'rg09_salinity']
# varname_qc  = [',1', '']
# vartitle    = 'Salinity, psu'

# varname     = ['temperature', 'rg09_temperature']
# varname_qc  = [',1', '']
# vartitle    = 'Temperature, degC'

# define name of the variable that includes levels for each collection
varname_levels = ['pressure',''] # for the gridded product, the level info is in the metadata, i.e. there is no variable in 'data' (for argo, 'pressure' is within the 'data' instead)

# levels for vertically integrated profiles
interp_levels        = list(range(10,2001))[0::20]

# in this example, we will use the 'box' selection (in the next cell),
# hence we indicate here the bottom/left and top/right vertices... 
# the other option is to search in a 'polygon' and indicate the polygon
# vertices in a list (first and last vertex should be the same)
regions_list         = [
                        [[-145.5,45.5],[-135.5,50.5]],
                        [[-50,45],[-40,50]],
                        ]
regions_list_type    = 'box'

regions_list_tags    = ['Pacific', 'Atlantic']

startDate            = '2021-01-01T00:00:00Z'
endDate              = '2021-12-31T00:00:00Z'

# box_regions_list     = [
#                         [[-78,45],[-10,50]],
#                         [[-180,45],[-127,50]],
#                         ]

In [ ]:
!date

data_reg = get_profiles_in_regions_and_horiz_ave(collections=collections,varname=varname,varname_qc=varname_qc,varname_levels=varname_levels,interp_levels=interp_levels,regions_list_source=regions_list,regions_list_source_type=regions_list_type,regions_list_source_tags=regions_list_tags,startDate=startDate,endDate=endDate,API_KEY=API_KEY)
            
!date    

In [ ]:
# plot the map for one timestep/level for each of the gridded products
for i,iaxr in enumerate(data_reg['regions_list_data_raw_xarray']):
    if iaxr:
        print(iaxr.sizes)
        print(data_reg['regions_list_tags'][i]+', '+data_reg['regions_list_collections'][i])
        plt.figure()
        iaxr['data'][:,:,0,0].plot()

In [ ]:
len(data_reg['regions_list_data_horiz_ave'])

In [ ]:
cols = ['k','r','b','m']

In [ ]:
# plot the horizontal average for (vertically) interpolated profiles and for the gridded product
plt.figure(figsize=(10,8))
for i,idata in enumerate(data_reg['regions_list_data_horiz_ave']):
    plt.plot(idata,data_reg['regions_list_data_horiz_ave_levels'][i],color=cols[i],linewidth=3)
plt.gca().invert_yaxis()
plt.ylabel('Vertical level (m or dbar)',size=16)
plt.xlabel(vartitle,size=16)
plt.legend([a_+', '+b_ for a_, b_ in zip(data_reg['regions_list_tags'],data_reg['regions_list_collections'])])


In [ ]:
# let's look at all the profiles that were vertically interpolated (except for the gridded products) to then compute the horizontal average above
plt.figure(figsize=(15,8))
for i,idata in enumerate(data_reg['regions_list_data_raw']):
    plt.subplot(1,len(data_reg['regions_list_data_horiz_ave']),i+1)    
    for iidata in idata:
        plt.plot(iidata[0],iidata[1],color=cols[i])
    plt.gca().invert_yaxis()
    if i==0:
        plt.ylabel('Vertical level (m or dbar)',size=16)
    plt.title(data_reg['regions_list_tags'][i]+', '+data_reg['regions_list_collections'][i])
    plt.xlabel(vartitle,size=16)
    

In [ ]:
# let's look at all the vertically interpolated profiles that were used to compute the horizontal average above
plt.figure(figsize=(15,8))
for i,idata in enumerate(data_reg['regions_list_data_vert_interp']):
    if idata:
        plt.subplot(1,len(data_reg['regions_list_data_horiz_ave']),i+1)    
        for iidata in idata:
            plt.plot(iidata,data_reg['regions_list_data_horiz_ave_levels'][i],color=cols[i])
        plt.gca().invert_yaxis()
        if i==0:
            plt.ylabel('Vertical level (m or dbar)',size=16)
        plt.title(data_reg['regions_list_tags'][i]+', '+data_reg['regions_list_collections'][i]+' (interpolated)')
        plt.xlabel(vartitle,size=16)
    

In [ ]:
#https://sites.google.com/view/paztronomer/blog/basic/python-colors
cols_months = ['deepskyblue', 'violet', 'firebrick', 'gold']
months      = [[12, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11]]
# for each product, plot the raw profiles color coded by month of the year
plt.figure(figsize=(15,8))
for i,idata in enumerate(data_reg['regions_list_data_raw']):
    itime = data_reg['regions_list_data_time'][i]
    
    plt.subplot(1,len(data_reg['regions_list_data_horiz_ave']),i+1)    
    for ii,iidata in enumerate(idata):
        for imm,mm in enumerate(months):
            if ii<=len(itime)-1 and itime[ii].month in mm:
                plt.plot(iidata[0],iidata[1],color=cols_months[imm])
    plt.gca().invert_yaxis()
    if i==0:
        plt.ylabel('Vertical level (m or dbar)',size=16)
    plt.title(data_reg['regions_list_tags'][i]+', '+data_reg['regions_list_collections'][i])
    plt.xlabel(vartitle,size=16)


In [ ]:
# bin in time? platform? woceline?

In [ ]:
# #regions_list_data_time_month = [x.month for y in regions_list_data_time for x in y]


# # plot raw profiles that go into the average colorcoded by month of year
# for i,idata in enumerate(regions_list_data_raw):
#     itime = regions_list_data_time[i]
#     for ii,iidata in enumerate(idata):
        
#         if ii<=len(itime)-1 and itime[ii].month in [6, 7, 8]:
#             plt.plot(iidata[0],iidata[1],c='k',marker='.')
            
#         # check for oxygen data
#         if ('doxy' in varname) and sum(np.array(iidata[0])<0) != 0:
#             check_data
# #plt.ylim([0, 20])
# plt.gca().invert_yaxis()        
# plt.ylabel('Vertical level')
# plt.xlabel(vartitle)